In [5]:
mkdir data

mkdir: cannot create directory ‘data’: File exists


In [6]:
!wget -q https://files.grouplens.org/datasets/movielens/ml-latest-small.zip -O data/ml-latest-small.zip

In [7]:
!unzip -q data/ml-latest-small.zip -d data

replace data/ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
import pandas as pd

In [10]:
ratings = pd.read_csv("data/ml-latest-small/ratings.csv")
movies = pd.read_csv("data/ml-latest-small/movies.csv")

In [11]:
df = pd.merge(ratings, movies, on='movieId')
df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Get Out (2017),Horror
100834,610,168252,5.0,1493846352,Logan (2017),Action|Sci-Fi


In [10]:
rating_matrix = df.pivot_table(index='userId', columns='title', values='rating')
rating_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [13]:
import numpy as np
from scipy.sparse import csr_matrix


R = rating_matrix.fillna(0).values
sparse_R = csr_matrix(R)

In [14]:
R

array([[0. , 0. , 0. , ..., 0. , 4. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4. , 0. , 0. , ..., 1.5, 0. , 0. ]])

# SVD with Surprise (Explicit MF)

In [23]:
!pip install "numpy<2.0.0" --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [ ]:
import os
os.kill(os.getpid(), 9)

: 

: 

: 

In [1]:
!pip install scikit-surprise

In [2]:
import surprise
print(surprise.__version__)

1.1.4


In [3]:
from surprise import SVD, Dataset, Reader

In [12]:
# Use Surprise's Reader to parse the dataframe
reader = Reader(rating_scale=(0.5, 5.0))  # rating scale in MovieLens
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Build trainset and testset (e.g., 80/20 split)
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [13]:
algo = SVD(n_factors=20, random_state=42)  # 20 latent factors, for example
algo.fit(trainset)


In [14]:
# Pick an example (user 1, movieId 50) from the testset
uid = str(1)       # Surprise expects user and item IDs as strings
mid = str(50)
pred = algo.predict(uid, mid)
print(f"Predicted rating for user {uid} on movie {mid}: {pred.est:.2f}")


Predicted rating for user 1 on movie 50: 3.50


In [15]:
from surprise import accuracy

# Generate predictions for the test set
predictions = algo.test(testset)

# Calculate RMSE and MAE
rmse = accuracy.rmse(predictions)  # Root Mean Squared Error
mae = accuracy.mae(predictions)    # Mean Absolute Error


RMSE: 0.8780
MAE:  0.6751


In [16]:
user_vec = algo.pu[0]   # latent factors for user with inner ID 0
item_vec = algo.qi[0]   # latent factors for item with inner ID 0
print(f"User0 factor vector: {user_vec}")
print(f"Item0 factor vector: {item_vec}")


User0 factor vector: [-0.0104593  -0.11075594  0.13106863  0.13560131 -0.0158802  -0.08846242
  0.29483349  0.12447541 -0.12562294  0.01805902  0.04168837 -0.03025305
 -0.15687714 -0.29228316 -0.20408676  0.18281409 -0.10066807 -0.11430069
 -0.11603668 -0.14721633]
Item0 factor vector: [ 0.04353494  0.15721122 -0.13495647  0.04001578  0.17305118  0.0587527
  0.13351926  0.08254423  0.09985935 -0.10105084 -0.1406025   0.01122602
 -0.23249771  0.08558504  0.12816424 -0.02335784 -0.07429933 -0.00382142
  0.0187307   0.01778445]


In [19]:
import numpy as np
pred_rating = np.dot(user_vec, item_vec)
pred_rating

-0.02484872347109904

# ALS (Alternating Least Squares)

In [20]:
# (This is a demonstration snippet - PySpark needed to run)
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

spark = SparkSession.builder.getOrCreate()
spark_df = spark.createDataFrame(df[['userId','movieId','rating']])  # convert pandas to Spark DataFrame

als = ALS(maxIter=10, regParam=0.1, rank=20, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop", seed=42)
model = als.fit(spark_df)

# Get predictions on test data (also a Spark DataFrame)
test_df = spark.createDataFrame(pd.DataFrame(testset, columns=['userId','movieId','rating']))
predictions = model.transform(test_df)
predictions.show(5)


+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   471|   2324|   5.0| 4.4054947|
|   463|   5378|   4.0| 3.5350997|
|   496| 106920|   4.5| 4.1630554|
|   148|  31658|   4.0| 3.6839943|
|   463|   1320|   4.0| 3.5027783|
+------+-------+------+----------+
only showing top 5 rows


# LightFM (Hybrid MF with implicit feedback)

In [24]:
!pip install lightfm-next

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00a 0:00:01


In [25]:
from lightfm import LightFM
from lightfm.data import Dataset

In [26]:
# Prepare data for LightFM
pos_ratings = df[df['rating'] >= 4.0]  # treat ratings 4 and 5 as positive
users = pos_ratings['userId'].unique()
items = pos_ratings['movieId'].unique()

dataset = Dataset()
dataset.fit(users, items)  # map user and item IDs to internal indices

# Build interactions matrix for training (and one for testing)
# Let's do a simple random split of interactions for implicit feedback
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(pos_ratings, test_size=0.2, random_state=42)

# Build interaction matrices
train_interactions, _ = dataset.build_interactions([(u, i) for u,i in zip(train_data['userId'], train_data['movieId'])])
test_interactions, _ = dataset.build_interactions([(u, i) for u,i in zip(test_data['userId'], test_data['movieId'])])
print(f"Train interactions: {train_interactions.getnnz()} positive entries, Test interactions: {test_interactions.getnnz()}")


Train interactions: 38864 positive entries, Test interactions: 9716


In [27]:
model = LightFM(no_components=20, loss='warp', random_state=42)
model.fit(train_interactions, epochs=10, num_threads=2)


In [28]:
from lightfm.evaluation import precision_at_k, recall_at_k

# Compute average precision@5 and recall@5 on test data
p_at_5 = precision_at_k(model, test_interactions, train_interactions, k=5).mean()
r_at_5 = recall_at_k(model, test_interactions, train_interactions, k=5).mean()
print(f"Precision@5: {p_at_5:.3f}, Recall@5: {r_at_5:.3f}")


Precision@5: 0.168, Recall@5: 0.078


In [29]:
user_embeddings = model.user_embeddings  
item_embeddings = model.item_embeddings
print(user_embeddings.shape, item_embeddings.shape)


(609, 20) (6298, 20)
